In [1]:
import numpy as np
import pandas
import copy
import os
from sklearn.utils import shuffle
import seaborn as sns
from tqdm import tqdm
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

from efficientnet_pytorch import EfficientNet

import sys  
sys.path.insert(0, './myCode')
from PLModel import PLModel
from CustomDataset import TotalDataset
from utils import loadModelCheckpoint

/data/cino/conda/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/data/cino/conda/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [3]:
IMAGE_SIZE = 224
gpus = [0]
BATCH_SIZE = 16 * len(gpus)
batches = round(512 / BATCH_SIZE)

In [4]:
skinDataset = []
labelName = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]

i = 0
#Reading the labels
df = pandas.read_csv("DataFrames/label.csv")
#shuffling the dataframe
df = shuffle(df, random_state=1234).reset_index(drop=True)

In [5]:
imageSizePerCNN = {
    "Resnext50" : 600,
    "Resnet152" : 600,
    "EfficientNetB7": 600,
    "EfficientNetB6": 528,
    "EfficientNetB5": 456,
    "EfficientNetB4": 380,
}

# Create Dataloader

In [6]:
dataset = TotalDataset(df)

## Models pythorch Lightning

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Models = []

cuda


In [8]:
model = PLModel('EfficientNetB4', EfficientNet.from_pretrained('efficientnet-b4', num_classes=8))
Models.append(model)

Loaded pretrained weights for efficientnet-b4
No loss specified, using default


In [9]:
model = PLModel('EfficientNetB5', EfficientNet.from_pretrained('efficientnet-b5', num_classes=8))
Models.append(model)

Loaded pretrained weights for efficientnet-b5
No loss specified, using default


In [10]:
model = PLModel('EfficientNetB6', EfficientNet.from_pretrained('efficientnet-b6', num_classes=8))
Models.append(model)

Loaded pretrained weights for efficientnet-b6
No loss specified, using default


In [11]:
resnext = torch.hub.load('pytorch/vision:v0.9.0', 'resnext50_32x4d', pretrained=True)
num_f = resnext.fc.in_features
resnext.fc = nn.Linear(num_f, 8)
Models.append(PLModel('Resnext50', resnext))

Using cache found in /data/cino/.cache/torch/hub/pytorch_vision_v0.9.0


No loss specified, using default


In [12]:
resnext = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
num_f = resnext.fc.in_features
resnext.fc = nn.Linear(num_f, 8)
Models.append(PLModel('Resnet152', resnext))

Using cache found in /data/cino/.cache/torch/hub/pytorch_vision_v0.9.0


No loss specified, using default


In [13]:
activation = {}

for model in Models:
    if model.name.find("EfficientNet") > -1:
        print(model.model._avg_pooling)
        model.model._avg_pooling.register_forward_hook(get_activation('avgpool'))
    else:
        print(model.model.avgpool)
        model.model.avgpool.register_forward_hook(get_activation('avgpool'))

AdaptiveAvgPool2d(output_size=1)
AdaptiveAvgPool2d(output_size=1)
AdaptiveAvgPool2d(output_size=1)
AdaptiveAvgPool2d(output_size=(1, 1))
AdaptiveAvgPool2d(output_size=(1, 1))


In [14]:
models_name = [model.name for model in Models ]
classes = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]
# Creating a MultiIndex for columns
columns = pandas.MultiIndex.from_product([models_name, classes], names=['Model', 'Label'])

# Create the DataFrame with MultiIndex columns and the image index
predictions = pandas.DataFrame(index=dataset.label.image, columns=columns)
activations = pandas.DataFrame(index=dataset.label.image, columns=columns)
hidden_layer = pandas.DataFrame(index=dataset.label.image, columns=models_name)

In [15]:
activations = pandas.read_csv("DataFrames/activations.csv",header=[0, 1], index_col = 0)
predictions = pandas.read_csv("DataFrames/probabilities.csv",header=[0, 1], index_col = 0)
hidden_layer = pandas.read_csv("DataFrames/last_hidden_layer.csv",header=[0], index_col = 0)

In [16]:
for model in Models:
    dataset = TotalDataset(df, imageSize = imageSizePerCNN[model.name])
    
    kFolds = dataset.getKFolds()
    for k, (_, validationDataset) in enumerate(kFolds):
        mypath = 'checkpoints/' + model.name + '_' + str(k) + 'Fold'

        print(mypath)
        ## Checking if the prediction is already done
        if not predictions.loc[validationDataset.label.image.values, (model.name, slice(None))].isna().any().any():
            continue

        #loading the model
        weights_paths = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
        weights_path = weights_paths[0]
        weights_path = mypath + "/" + weights_path
        model = loadModelCheckpoint(weights_path, model)
        
        model.eval()        

        #Calculate the predictions
        for index, image_name in tqdm(zip(validationDataset.label.index, validationDataset.label.image.values), 
                                       total=len(validationDataset.label.index), 
                                       desc=f'Validating {model.name} - Fold {k}'):
            image, _ = validationDataset[index]
            output = model(image.unsqueeze(0))
            probabilities = F.softmax(output, dim=1)
            last_cnn_hidden = json.dumps(np.array(activation['avgpool'].to('cpu')).reshape(-1).tolist())
            activations.loc[image_name, (model.name, slice(None))] = output.detach().numpy()
            predictions.loc[image_name, (model.name, slice(None))] = probabilities.detach().numpy()
            hidden_layer.loc[image_name, model.name] = last_cnn_hidden

        #saving the results
        predictions.to_csv('DataFrames/probabilities.csv')
        activations.to_csv('DataFrames/activations.csv')
        hidden_layer.to_csv('DataFrames/last_hidden_layer.csv')

checkpoints/EfficientNetB4_0Fold
checkpoints/EfficientNetB4_1Fold
checkpoints/EfficientNetB4_2Fold
checkpoints/EfficientNetB4_3Fold
checkpoints/EfficientNetB4_4Fold
checkpoints/EfficientNetB5_0Fold
checkpoints/EfficientNetB5_1Fold
checkpoints/EfficientNetB5_2Fold
checkpoints/EfficientNetB5_3Fold
checkpoints/EfficientNetB5_4Fold
checkpoints/EfficientNetB6_0Fold
checkpoints/EfficientNetB6_1Fold
checkpoints/EfficientNetB6_2Fold
checkpoints/EfficientNetB6_3Fold
checkpoints/EfficientNetB6_4Fold
checkpoints/Resnext50_0Fold
checkpoints/Resnext50_1Fold
checkpoints/Resnext50_2Fold
checkpoints/Resnext50_3Fold
Model Loaded


Validating Resnext50 - Fold 3: 100%|████████| 5067/5067 [24:48<00:00,  3.40it/s]


checkpoints/Resnext50_4Fold
Model Loaded


Validating Resnext50 - Fold 4: 100%|████████| 5059/5059 [25:38<00:00,  3.29it/s]


checkpoints/Resnet152_0Fold
Model Loaded


...

Validating Resnet152 - Fold 2: 100%|████████| 5067/5067 [41:06<00:00,  2.05it/s]


checkpoints/Resnet152_3Fold
Model Loaded


Validating Resnet152 - Fold 3: 100%|████████| 5067/5067 [39:20<00:00,  2.15it/s]


checkpoints/Resnet152_4Fold
Model Loaded


Validating Resnet152 - Fold 4: 100%|████████| 5059/5059 [39:19<00:00,  2.14it/s]


In [17]:
predictions.to_csv('DataFrames/probabilities.csv')
activations.to_csv('DataFrames/activations.csv')
hidden_layer.to_csv('DataFrames/last_hidden_layer.csv')